<a href="https://colab.research.google.com/github/Nilanshrajput/Video_Generation_Transformer/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import os
from collections import OrderedDict
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import init
import functools
import torchvision.models as models

from __future__ import absolute_import, division, print_function, unicode_literals


from torchvision import models
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torchvision import transforms, utils
import matplotlib.pyplot as plt

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import re

import argparse
import warnings
from datetime import datetime
from glob import glob
from shutil import copyfile


from torch import autograd
from torch.nn import functional as F
import torch.nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed

In [0]:
import os
import re
import math
from pathlib import Path
# Suppose you are trying to load pre-trained resnet model in directory- models\resnet

os.environ['TORCH_HOME'] = 'D:\dev\Pytorch_Models\models\\resnet' #setting the environment variable


In [27]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [48]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 134923 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [71]:
home ="drive/dl_projects/Video_Genaeration/output"
home = Path(home)
import tensorflow

In [56]:
!ls "drive/dl_projects/Video_Genaeration/"

img.jpg  Model1  output  Pix2Pix.ipynb	Transformer.ipynb  Video_Gen


In [1]:
!wget --header="Host: www.cityscapes-dataset.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.cityscapes-dataset.com/downloads/" --header="Cookie: PHPSESSID=4f0gk1d5hbc0ikttngvpm8ff52" --header="Connection: keep-alive" "https://www.cityscapes-dataset.com/file-handling/?packageID=4" -O "leftImg8bit_trainextra.zip" -c

--2019-11-20 22:13:22--  https://www.cityscapes-dataset.com/file-handling/?packageID=4
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47230218747 (44G) [application/octet-stream]
Saving to: ‘leftImg8bit_trainextra.zip’

leftImg8bit_trainex 100%[===================>]  43.99G  98.2MB/s    in 7m 26s  

2019-11-20 22:20:53 (101 MB/s) - ‘leftImg8bit_trainextra.zip’ saved [47230218747/47230218747]



In [0]:
!unzip leftImg8bit_trainextra.zip

In [0]:

import json
import os
from collections import namedtuple
import zipfile

from PIL import Image


class CityscapeDataset(Dataset):
    """`Cityscapes <http://www.cityscapes-dataset.com/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory ``leftImg8bit``
            
        split (string, optional): The image split to use, ``train``, ``train_extra`` or ``val``
       
        transform (callable, optional): A function/transform that takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``  
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.

    Examples:

      
    """

    # Based on https://github.com/mcordts/cityscapesScripts
  

    def __init__(self, root=None, split='train_extra', transforms=None):
        if root is not None:
          self.images_dir = os.path.join(self.root, 'leftImg8bit', split)
        else:
          self.images_dir = os.path.join('leftImg8bit', split)
        self.split = split
        self.images = []
        self.root = root
        self.transforms = transforms
  
        #valid_modes = ("train", "train_extra", "val")
        
        for city in os.listdir(self.images_dir):
            img_dir = os.path.join(self.images_dir, city)
            images_city=[]
            for file_name in os.listdir(img_dir):
                
                images_city.append(os.path.join(img_dir, file_name))
            # re.split(\d+,input) splits by integer value
            images_city.sort( key= lambda text: int(re.split('(\d+)',text)[3]+re.split('(\d+)',text) [5] ))
            self.images+=images_city
        
    def paths_dir(self):
      return(self.images)

    def im_path(self):
      return self.images            
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
             image 
        """

        image = Image.open(self.images[index]).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        return image


    def __len__(self):
        return len(self.images)



In [0]:

import json
import os
from collections import namedtuple
import zipfile

from PIL import Image


class CityscapeDataset(Dataset):
    """`Cityscapes <http://www.cityscapes-dataset.com/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory ``leftImg8bit``
            
        split (string, optional): The image split to use, ``train``, ``train_extra`` or ``val``
       
        transform (callable, optional): A function/transform that takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``  
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.

    Examples:

      
    """

    # Based on https://github.com/mcordts/cityscapesScripts
  

    def __init__(self, root=None, split='train_extra', transforms=None):
        if root is not None:
          self.images_dir = os.path.join(self.root, 'leftImg8bit', split)
        else:
          self.images_dir = os.path.join('leftImg8bit', split)
        self.split = split
        self.images = []
        self.root = root
        self.transforms = transforms
  
        #valid_modes = ("train", "train_extra", "val")
        
        for city in os.listdir(self.images_dir):
            img_dir = os.path.join(self.images_dir, city)
            images_city=[]
            for file_name in os.listdir(img_dir):
                
                images_city.append(os.path.join(img_dir, file_name))
            # re.split(\d+,input) splits by integer value
            images_city.sort( key= lambda text: int(re.split('(\d+)',text)[3]+re.split('(\d+)',text) [5] ))
            self.images+=images_city
        
    def print_dir(self):
      print(self.images)

    def im_path(self):
      return self.images            
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
             image 
        """

        image = Image.open(self.images[index]).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        return image


    def __len__(self):
        return len(self.images)



In [0]:
scaler = transforms.Resize((256, 256))
to_tensor = transforms.ToTensor()

In [0]:
#not adding normalization as image looses textures and clrs for recostruction

composed_transforms=transforms.Compose([scaler,to_tensor])

In [0]:
transformed_dataset = CityscapeDataset(
                                           transforms=composed_transforms
                                           )

In [0]:
#x=next(iter(dataloader))

In [0]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-5])
        
    def forward(self, x):
        x = self.features(x)
        return x

In [0]:
#res50_model = models.resnet50(pretrained=True)
#res50_conv2 = ResNet50Bottom(res50_model).cuda()
#
#outputs = res50_conv2(x.to('cuda'))
#outputs.data.shape

In [0]:
class FeatureChanger(nn.Module):
    def __init__(self,in_channels, out_channels):
        super().__init__()
        
        self.kernel_size = 1

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv2d_0 = torch.nn.Conv2d(in_channels, out_channels, self.kernel_size, stride=1)
        #self.con2d_1 = torch.nn.Conv2d(out_channels*2, out_channels, kernel_size, stride=1)
        
    def forward(self, batch):
        x = self.conv2d_0(batch)
        #x = self.con2d_1(x)
        return x

In [0]:
class MaskedSelfAttention(nn.Module):

  def __init__(self,heads, d_model, dropout= .4):
    super().__init__()

    self.d_model = d_model
    self.heads = heads
    self.soft2d = nn.Softmax2d()
    self.drop = nn.Dropout(dropout)
    

  def _generate_square_subsequent_mask(self, sz):
    #sz= batch size
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

  def mult_4d(self, batch, q_w):
    bs= batch.shape[0]
    layers = batch.shape[1]
    dim1 = batch.shape[2]
    dim2 = batch.shape[3]
    output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
    return output.view(bs,layers,dim1,dim2)




  def forward(self, q , k, v):

    #score
    attn_output_weights = self.mult_4d(q, k.transpose(-2,-1))/math.sqrt(self.d_model/2)
    soft_scores = self.soft2d(attn_output_weights)
    soft_scores = self.drop(soft_scores).to('cuda')
    output = self.mult_4d(v, soft_scores)

    return output

In [0]:

class MultiHeadAttention(nn.Module):
    def __init__(self, bs, heads, d_model, dim1, dim2, dropout = 0.4):
        super().__init__()
        
        self.d_model = d_model
        self.h = heads
        self.dim1 = dim1
        self.dim2 =dim2
        self.dropout = dropout
        self.nu_feat =d_model # int(d_model/self.h*2)
        self.bs = bs
        res50_model = models.resnet50(pretrained=True)
        self.resnet_back = ResNet50Bottom(res50_model).cuda()


        self.q_3d = torch.nn.Parameter(torch.empty(bs,self.nu_feat,dim1,dim2,requires_grad=True))
        self.k_3d = torch.nn.Parameter(torch.empty(bs,self.nu_feat,dim1,dim2,requires_grad=True))
        self.v_3d = torch.nn.Parameter(torch.empty(bs,self.nu_feat,dim1,dim2,requires_grad=True))
   

        nn.init.kaiming_normal_(self.q_3d, mode='fan_in')
        nn.init.kaiming_normal_(self.k_3d, mode='fan_in')
        nn.init.kaiming_normal_(self.v_3d, mode='fan_in')

        self.attention_layer= MaskedSelfAttention(self.h,d_model,dropout)
    
    def mult_4d(self, batch, q_w, bs):
      
      layers = batch.shape[1]
      dim1 = batch.shape[2]
      dim2 = batch.shape[3]
      #print(layers, dim1,dim2,bs)
      #print(q_w.shape)
      output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
      return output.view(bs,layers,dim1,dim2)

    def forward(self, batch, mask=None):
        bs = batch.size(0)
        batch = batch.to('cuda')
        """ 
        batch = self.resnet_back(batch.to('cuda'))        
        channels = batch.shape[1]
        feature_layer1 = FeatureChanger(channels,int(channels/self.h*2)).cuda()
        batch = feature_layer1(batch)

        """

        q = self.mult_4d(batch,self.q_3d.to('cuda'),bs)
        k = self.mult_4d(batch,self.k_3d.to('cuda'),bs)
        v = self.mult_4d(batch,self.v_3d.to('cuda'),bs)

        """
        #divide feature space in no of heads each head works on certain features only
        #returns tuple of size=heads

        batch_chunks= torch.chunk(batch, heads, dim=1)
        #converting tuple to tensor
        batch_chunks = torch.stack(batch_chunks, dim =0)
        """

        attention_out = self.attention_layer( q, k, v)
        #feature_layer2 = FeatureChanger(int(channels/self.h*2),channels).cuda()
        #output = feature_layer2(batch)
        output = attention_out
  
    
        return output

In [0]:
batch_size = 2
dataloader = DataLoader(transformed_dataset, batch_size,
                        sampler=torch.utils.data.SequentialSampler(transformed_dataset) , num_workers=4)

In [17]:
"""
heads = 8
d_model = 3
dim1 = dim2= 224
dropout = 0.4


MHA= MultiHeadAttention(batch_size, heads, d_model, dim1, dim2, dropout = 0.4)

"""


'\nheads = 8\nd_model = 3\ndim1 = dim2= 224\ndropout = 0.4\n\n\nMHA= MultiHeadAttention(batch_size, heads, d_model, dim1, dim2, dropout = 0.4)\n\n'

In [0]:

#output = MHA(next(iter(dataloader)))

In [0]:
#output.shape

In [0]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#
class Generator(nn.Module):
    def __init__(self, img_size_min, num_scale, size_list, bs, heads, d_model, dim1, dim2, dropout = 0.4, scale_factor=4/3):
        super(Generator, self).__init__()
        self.img_size_min = img_size_min
        self.scale_factor = scale_factor
        self.num_scale = num_scale
        self.nf = 32
        self.current_scale = 0
        self.d_model = d_model
        self.h = heads
        self.dim1 = dim1
        self.dim2 =dim2
        self.dropout = dropout
        self.nu_feat =d_model # int(d_model/self.h*2)
        self.bs = bs

        self.size_list = size_list #[int(self.img_size_min * scale_factor**i) for i in range(num_scale + 1)]
        print(self.size_list)
        self.MHA= MultiHeadAttention(bs, heads, d_model, dim1, dim2, dropout )
        self.sub_generators = nn.ModuleList()

        first_generator = nn.ModuleList()

        first_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                             nn.BatchNorm2d(self.nf),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                             nn.Tanh()))

        first_generator = nn.Sequential(*first_generator)

        self.sub_generators.append(first_generator)

    def forward(self, batch, img=None):
        x_list = []
        #print(batch[0].shape)
        dim2 = dim1 = batch[0].shape[-1]
        #self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
        #x_first = self.sub_generators[0](self.MHA(batch[0]))
        #print(x_first.size())
        x_first = self.sub_generators[0](batch[0])
        x_list.append(x_first)

        if img is not None:
            x_inter = img
        else:
            x_inter = x_first

        for i in range(1, self.current_scale + 1):
            x_inter = F.interpolate(x_inter, (self.size_list[i], self.size_list[i]), mode='bilinear', align_corners=True)
            #print(x_inter.size())
            x_prev = x_inter
            x_inter = F.pad(x_inter, [5, 5, 5, 5], value=0)
            #print(self.current_scale)
            dim2 = dim1 = batch[i].shape[-1]
            """
            self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
            x_inter = x_inter + self.MHA(batch[i])
            """
            x_inter = x_inter + (batch[i])
            x_inter = self.sub_generators[i](x_inter) + x_prev
            x_list.append(x_inter)
        #print(self.size_list)
        return x_list

    def progress(self):
        self.current_scale += 1

        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_generator = nn.ModuleList()
        tmp_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                           nn.BatchNorm2d(self.nf),
                                           nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                               nn.BatchNorm2d(self.nf),
                                               nn.LeakyReLU(2e-1)))

        tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                           nn.Tanh()))

        tmp_generator = nn.Sequential(*tmp_generator)

        if self.current_scale % 4 != 0:
            prev_generator = self.sub_generators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_generator.load_state_dict(prev_generator.state_dict())

        self.sub_generators.append(tmp_generator)
        print("GENERATOR PROGRESSION DONE")

In [0]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.nf = 32
        self.current_scale = 0

        self.sub_discriminators = nn.ModuleList()

        first_discriminator = nn.ModuleList()

        first_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        first_discriminator = nn.Sequential(*first_discriminator)

        self.sub_discriminators.append(first_discriminator)

    def forward(self, x):
        out = self.sub_discriminators[self.current_scale](x)
        return out

    def progress(self):
        self.current_scale += 1
        # Lower scale discriminators are not used in later ... replace append to assign?
        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_discriminator = nn.ModuleList()
        tmp_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                               nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                   nn.BatchNorm2d(self.nf),
                                                   nn.LeakyReLU(2e-1)))

        tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        tmp_discriminator = nn.Sequential(*tmp_discriminator)

        if self.current_scale % 4 != 0:
            prev_discriminator = self.sub_discriminators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_discriminator.load_state_dict(prev_discriminator.state_dict())

        self.sub_discriminators.append(tmp_discriminator)
        print("DISCRIMINATOR PROGRESSION DONE")

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_grad_gp(d_out, x_in):
    batch_size = x_in.size(0)
    grad_dout = autograd.grad(
        outputs=d_out.sum(), inputs=x_in,grad_outputs=torch.ones_like(d_out.sum()),
        create_graph=True, retain_graph=True, only_inputs=True)[0]
    grad_dout2 = grad_dout.pow(2)
    assert(grad_dout2.size() == x_in.size())
    reg = grad_dout2.view(batch_size, -1).sum(1)
    return reg


def compute_grad_gp_wgan(D, x_real, x_fake, gpu):
    alpha = torch.rand(x_real.size(0), 1, 1, 1).cuda(gpu)

    x_interpolate = ((1 - alpha) * x_real + alpha * x_fake).detach()
    x_interpolate.requires_grad = True
    d_inter_logit = D(x_interpolate)
    grad = torch.autograd.grad(d_inter_logit, x_interpolate,
                               grad_outputs=torch.ones_like(d_inter_logit), create_graph=True)[0]

    norm = grad.view(grad.size(0), -1).norm(p=2, dim=1)

    d_gp = ((norm - 1) ** 2).mean()
    return d_gp

In [0]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#

from tqdm import trange
import torchvision.utils as vutils


def trainSinGAN(data_loader, networks, opts, stage, args, additional):
  # avg meter
  d_losses = AverageMeter()
  g_losses = AverageMeter()

  # set nets
  D = networks[0]
  G = networks[1]
  # set opts
  d_opt = opts['d_opt']
  g_opt = opts['g_opt']
  # switch to train mode
  D.train()
  G.train()
  # summary writer
  # writer = additional[0]
  train_it = iter(data_loader)
  # total_iter = 2000 * (args.num_scale - stage + 1)
  # decay_lr = 1600 * (args.num_scale - stage + 1)
  total_iter = 2000
  decay_lr = 4

  d_iter = 3
  g_iter = 3

  t_train = trange(0, total_iter, initial=0, total=total_iter)
  x_in = next(train_it)

  x_in = x_in.cuda( non_blocking=True)
  x_org = x_in
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  #x_0 = F.interpolate(x_0, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  
  

  """
  z_rec = additional['z_rec']

  for z_idx in range(len(z_rec)):
      z_rec[z_idx] = z_rec[z_idx].cuda(args.gpu, non_blocking=True)

  x_in = next(train_it)

  x_in = x_in.cuda(args.gpu, non_blocking=True)
  x_org = x_in
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)
  vutils.save_image(x_in.detach().cpu(), os.path.join(args.res_dir, 'ORGTRAIN_{}.png'.format(stage)),
                    nrow=1, normalize=True)
   """
  x_in_list = []

  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      x_tmp = F.pad(x_tmp, [5, 5, 5, 5], value=0)
      x_in_list.append(x_tmp)
  
  x_in_list_nopad = []
  
  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      
      x_in_list_nopad.append(x_tmp)
  z_rec = F.pad(x_in, [5, 5, 5, 5], value=0)
  for i in t_train:
      if i == decay_lr:
          for param_group in d_opt.param_groups:
                  param_group['lr'] *= 0.1
                  print("DISCRIMINATOR LEARNING RATE UPDATE TO :", param_group['lr'])
          for param_group in g_opt.param_groups:
                  param_group['lr'] *= 0.1
                  print("GENERATOR LEARNING RATE UPDATE TO :", param_group['lr'])

      for _ in range(g_iter):
          g_opt.zero_grad()
          #print(type(z_rec))
          x_rec_list = G(x_in_list)
          #print(x_rec_list[-1].shape)
          #print(x_in.shape)
          g_rec = F.mse_loss(x_rec_list[-1], x_in)
          # calculate rmse for each scale
          rmse_list = [1.0]
          for rmseidx in range(1, stage + 1):
              rmse = torch.sqrt(F.mse_loss(x_rec_list[rmseidx], x_in_list_nopad[rmseidx]))
              rmse_list.append(rmse)

          z_list = [F.pad(rmse_list[z_idx] * torch.randn(args.batch_size, 3, args.size_list[z_idx],
                                              args.size_list[z_idx]).cuda(args.gpu, non_blocking=True),
                          [5, 5, 5, 5], value=0) for z_idx in range(stage + 1)]
          
          x_fake_list = G(z_list )
          #print(len(x_fake_list[-1]))

          g_fake_logit = D(x_fake_list[-1])

          ones = torch.ones_like(g_fake_logit).cuda(args.gpu)

          if args.gantype == 'wgangp':
              # wgan gp
              g_fake = -torch.mean(g_fake_logit, (2, 3))
              g_loss = g_fake + 10.0 * g_rec
          elif args.gantype == 'zerogp':
              # zero centered GP
              g_fake = F.binary_cross_entropy_with_logits(g_fake_logit, ones, reduction='none').mean()
              g_loss = g_fake + 100.0 * g_rec

          elif args.gantype == 'lsgan':
              # lsgan
              g_fake = F.mse_loss(torch.mean(g_fake_logit, (2, 3)), 0.9 * ones)
              g_loss = g_fake + 50.0 * g_rec

          g_loss.backward(retain_graph = True)
          g_opt.step()

          g_losses.update(g_loss.item(), x_in.size(0))

      # Update discriminator
      for _ in range(d_iter):
          x_in.requires_grad = True

          d_opt.zero_grad()
          x_fake_list =  G(z_list )
          

          d_fake_logit = g_fake_logit
          d_real_logit = D(x_in)

          ones = torch.ones_like(d_real_logit).cuda(args.gpu)
          zeros = torch.zeros_like(d_fake_logit).cuda(args.gpu)

          if args.gantype == 'wgangp':
              # wgan gp
              d_fake = torch.mean(d_fake_logit, (2, 3))
              d_real = -torch.mean(d_real_logit, (2, 3))
              d_gp = compute_grad_gp_wgan(D, x_in, x_fake_list[-1], args.gpu)
              d_loss = d_real + d_fake + 0.1 * d_gp
          elif args.gantype == 'zerogp':
              # zero centered GP
              # d_fake = F.binary_cross_entropy_with_logits(torch.mean(d_fake_logit, (2, 3)), zeros)
              d_fake = F.binary_cross_entropy_with_logits(d_fake_logit, zeros, reduction='none').mean()
              # d_real = F.binary_cross_entropy_with_logits(torch.mean(d_real_logit, (2, 3)), ones)
              d_real = F.binary_cross_entropy_with_logits(d_real_logit, ones, reduction='none').mean()
              #d_gp = compute_grad_gp(torch.mean(d_real_logit, (2, 3)), x_in)
              d_loss = d_real + d_fake# + 10.0 * d_gp

          elif args.gantype == 'lsgan':
              # lsgan
              d_fake = F.mse_loss(torch.mean(d_fake_logit, (2, 3)), zeros)
              d_real = F.mse_loss(torch.mean(d_real_logit, (2, 3)), 0.9 * ones)
              d_loss = d_real + d_fake

          d_loss.backward(retain_graph=True)
          d_opt.step()

          d_losses.update(d_loss.item(), x_in.size(0))

      t_train.set_description('Stage: [{}/{}] Avg Loss: D[{d_losses.avg:.3f}] G[{g_losses.avg:.3f}] RMSE[{rmse:.3f}]'
                              .format(stage, args.num_scale, d_losses=d_losses, g_losses=g_losses, rmse=rmse_list[-1]))

In [0]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [0]:
class Arguments():
  def __init__(self,gpu = None,gantype = 'zerogp',model_name = 'SinGan', batch_size = 128, img_size_max= 224, img_size_min=20,
               load_model = None, multiprocessing_distributed= None, world_size=1, log_dir= None, res_dir = None, num_scale = None):
    self.gpu = gpu
    self.gantype = gantype
    self.model_name = "SinGan"
    self.batch_size = batch_size
    self.img_size_max= img_size_max
    self.img_size_min=img_size_min
    self.load_model = load_model
    self.multiprocessing_distributed= multiprocessing_distributed 
    self.world_size=world_size
    self.log_dir= log_dir
    self.res_dir = res_dir 
    self.num_scale = num_scale 
    self.stage = None
    self.distributed = None
    self.workers = 8
    self.d_model = None
    self.h = None
    self.dim1 = None
    self.dim2 = None
    self.dropout = None
    self.nu_feat =None
    self.dataset = None
    



In [0]:

def save_checkpoint(state, check_list, log_dir, epoch=0):
    check_file = os.path.join(log_dir, 'model_{}.ckpt'.format(epoch))
    torch.save(state, check_file)
    check_list.write('model_{}.ckpt\n'.format(epoch))

def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
def formatted_print(notice, value):
    print('{0:<40}{1:<40}'.format(notice, value))



In [0]:

def main_worker(gpu, ngpus_per_node, args):
    args.gpu = 0


    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))
    args.distributed = False
    if args.distributed:
        if args.multiprocessing_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend='nccl', init_method='tcp://127.0.0.1:'+args.port,
                                world_size=args.world_size, rank=args.rank)

    ################
    # Define model #
    ################
    # 4/3 : scale factor in the paper
    scale_factor = 4/3



    tmp_scale = args.img_size_max / args.img_size_min
    args.num_scale = int(np.round(np.log(tmp_scale) / np.log(scale_factor)))
    args.size_list = [int(args.img_size_min * scale_factor**i) for i in range(args.num_scale + 1)]
    #args.size_list = [int(args.img_size_min *i) for i in range(1,args.num_scale + 1)]

    discriminator = Discriminator()
    generator = Generator(args.img_size_min, args.num_scale, args.size_list, args.batch_size, args.heads, args.d_model, args.dim1, args.dim2, args.dropout,  scale_factor)

    networks = [discriminator, generator]

    if args.distributed:
        if args.gpu is not None:
            print('Distributed to', args.gpu)
            torch.cuda.set_device(args.gpu)
            networks = [x.cuda(args.gpu) for x in networks]
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int(args.workers / ngpus_per_node)
            networks = [torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu) for x in networks]
        else:
            networks = [x.cuda() for x in networks]
            networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        networks = [x.cuda(args.gpu) for x in networks]
    else:
        networks = [torch.nn.DataParallel(x).cuda() for x in networks]

    discriminator, generator, = networks

    ######################
    # Loss and Optimizer #
    ######################
    if args.distributed:
        d_opt = torch.optim.Adam(discriminator.module.sub_discriminators[0].parameters(), 5e-4, (0.5, 0.999))
        g_opt = torch.optim.Adam(generator.module.sub_generators[0].parameters(), 5e-4, (0.5, 0.999))
    else:
        d_opt = torch.optim.Adam(discriminator.sub_discriminators[0].parameters(), 5e-4, (0.5, 0.999))
        g_opt = torch.optim.Adam(generator.sub_generators[0].parameters(), 5e-4, (0.5, 0.999))

    ##############
    # Load model #
    ##############
    args.stage = 0
    if args.load_model is not None:
        check_load = open(os.path.join(args.log_dir, "checkpoint.txt"), 'r')
        to_restore = check_load.readlines()[-1].strip()
        load_file = os.path.join(args.log_dir, to_restore)
        if os.path.isfile(load_file):
            print("=> loading checkpoint '{}'".format(load_file))
            checkpoint = torch.load(load_file, map_location='cpu')
            for _ in range(int(checkpoint['stage'])):
                generator.progress()
                discriminator.progress()
            networks = [discriminator, generator]
            if args.distributed:
                if args.gpu is not None:
                    print('Distributed to', args.gpu)
                    torch.cuda.set_device(args.gpu)
                    networks = [x.cuda(args.gpu) for x in networks]
                    args.batch_size = int(args.batch_size / ngpus_per_node)
                    args.workers = int(args.workers / ngpus_per_node)
                    networks = [
                        torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu) for
                        x in networks]
                else:
                    networks = [x.cuda() for x in networks]
                    networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

            elif args.gpu is not None:
                torch.cuda.set_device(args.gpu)
                networks = [x.cuda(args.gpu) for x in networks]
            else:
                networks = [torch.nn.DataParallel(x).cuda() for x in networks]

            discriminator, generator, = networks

            args.stage = checkpoint['stage']
            args.img_to_use = checkpoint['img_to_use']
            discriminator.load_state_dict(checkpoint['D_state_dict'])
            generator.load_state_dict(checkpoint['G_state_dict'])
            d_opt.load_state_dict(checkpoint['d_optimizer'])
            g_opt.load_state_dict(checkpoint['g_optimizer'])
            print("=> loaded checkpoint '{}' (stage {})"
                  .format(load_file, checkpoint['stage']))
        else:
            print("=> no checkpoint found at '{}'".format(args.log_dir))

    cudnn.benchmark = True

    ###########
    # Dataset #
    ###########
    transformed_dataset = CityscapeDataset(
                                           transforms=composed_transforms
                                           )

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = DataLoader(transformed_dataset, batch_size = args.batch_size,
                            sampler=torch.utils.data.SequentialSampler(transformed_dataset) ,  num_workers=args.workers)
  

    ######################
    # Validate and Train #
    ######################
    z_fix_list = [F.pad(torch.randn(args.batch_size, 3, args.size_list[0], args.size_list[0]), [5, 5, 5, 5], value=0)]
    zero_list = [F.pad(torch.zeros(args.batch_size, 3, args.size_list[zeros_idx], args.size_list[zeros_idx]),
                       [5, 5, 5, 5], value=0) for zeros_idx in range(1, args.num_scale + 1)]
    z_fix_list = z_fix_list + zero_list
    """
    if args.validation:
        validateSinGAN(train_loader, networks, args.stage, args, {"z_rec": z_fix_list})
        return

    elif args.test:
        validateSinGAN(train_loader, networks, args.stage, args, {"z_rec": z_fix_list})
        return
    """

    if not args.multiprocessing_distributed or (args.multiprocessing_distributed and args.rank % ngpus_per_node == 0):
        check_list = open(os.path.join(args.log_dir, "checkpoint.txt"), "a+")
        record_txt = open(os.path.join(args.log_dir, "record.txt"), "a+")
        record_txt.write('DATASET\t:\t{}\n'.format(args.dataset))
        record_txt.write('GANTYPE\t:\t{}\n'.format(args.gantype))
#        record_txt.write('IMGTOUSE\t:\t{}\n'.format(args.img_to_use))
        record_txt.close()

    for stage in range(args.stage, args.num_scale + 1):
        if args.distributed:
            train_sampler.set_epoch(stage)

        trainSinGAN(train_loader, networks, {"d_opt": d_opt, "g_opt": g_opt}, stage, args, {"z_rec": z_fix_list})
       # validateSinGAN(train_loader, networks, stage, args, {"z_rec": z_fix_list})

        if args.distributed:
            discriminator.module.progress()
            generator.module.progress()
        else:
            discriminator.progress()
            generator.progress()

        networks = [discriminator, generator]

        if args.distributed:
            if args.gpu is not None:
                print('Distributed', args.gpu)
                torch.cuda.set_device(args.gpu)
                networks = [x.cuda(args.gpu) for x in networks]
                args.batch_size = int(args.batch_size / ngpus_per_node)
                args.workers = int(args.workers / ngpus_per_node)
                networks = [torch.nn.parallel.DistributedDataParallel(x, device_ids=[args.gpu], output_device=args.gpu)
                            for x in networks]
            else:
                networks = [x.cuda() for x in networks]
                networks = [torch.nn.parallel.DistributedDataParallel(x) for x in networks]

        elif args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            networks = [x.cuda(args.gpu) for x in networks]
        else:
            networks = [torch.nn.DataParallel(x).cuda() for x in networks]

        discriminator, generator, = networks

        # Update the networks at finest scale
        if args.distributed:
            for net_idx in range(generator.module.current_scale):
                for param in generator.module.sub_generators[net_idx].parameters():
                    param.requires_grad = False
                for param in discriminator.module.sub_discriminators[net_idx].parameters():
                    param.requires_grad = False

            d_opt = torch.optim.Adam(discriminator.module.sub_discriminators[discriminator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
            g_opt = torch.optim.Adam(generator.module.sub_generators[generator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
        else:
            for net_idx in range(generator.current_scale):
                for param in generator.sub_generators[net_idx].parameters():
                    param.requires_grad = False
                for param in discriminator.sub_discriminators[net_idx].parameters():
                    param.requires_grad = False

            d_opt = torch.optim.Adam(discriminator.sub_discriminators[discriminator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))
            g_opt = torch.optim.Adam(generator.sub_generators[generator.current_scale].parameters(),
                                     5e-4, (0.5, 0.999))

        ##############
        # Save model #
        ##############
        if not args.multiprocessing_distributed or (args.multiprocessing_distributed and args.rank % ngpus_per_node == 0):
            if stage == 0:
                check_list = open(os.path.join(args.log_dir, "checkpoint.txt"), "a+")
            save_checkpoint({
                'stage': stage + 1,
                'D_state_dict': discriminator.state_dict(),
                'G_state_dict': generator.state_dict(),
                'd_optimizer': d_opt.state_dict(),
                'g_optimizer': g_opt.state_dict()
                
            }, check_list, args.log_dir, stage + 1)
            if stage == args.num_scale:
                check_list.close()

    return generator


In [61]:
args = Arguments()
args.gpu=0
args.distributed = False
args.load_model = None
args.batch_size = 2
args.img_size_max =  224
args.img_size_min = 28
args.heads = 8
args.d_model = 3
args.nu_feat =3
args.dim1 = 224
args.dim2 = 224
args.dropout = 0.4 
args.dataset = "train"

if args.gpu is not None:
   
    warnings.warn('You have chosen a specific GPU. This will completely '
                  'disable data parallelism.')
"""
args.distributed = args.world_size > 1 or args.multiprocessing_distributed


"""
ngpus_per_node = torch.cuda.device_count()
args.load_model = None
if args.load_model is None:
    args.model_name = '{}_{}'.format(args.model_name, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
else:
    args.model_name = args.load_model

makedirs(home/'logs')
makedirs(home/'results')

args.log_dir = os.path.join(home,'./logs', args.model_name)
args.res_dir = os.path.join(home,'./results', args.model_name)

makedirs(args.log_dir)
makedirs(os.path.join(args.log_dir, 'codes'))
makedirs(os.path.join(args.log_dir, 'codes', 'models'))
makedirs(args.res_dir)

if args.load_model is None:
    pyfiles = glob("./*.py")
    modelfiles = glob('./models/*.py')
    for py in pyfiles:
        copyfile(py, os.path.join(args.log_dir, 'codes') + "/" + py)
    for py in modelfiles:
        copyfile(py, os.path.join(args.log_dir, 'codes', py[2:]))

formatted_print('Total Number of GPUs:', ngpus_per_node)
formatted_print('Total Number of Workers:', args.workers)
formatted_print('Batch Size:', args.batch_size)
formatted_print('Max image Size:', args.img_size_max)
formatted_print('Min image Size:', args.img_size_min)
formatted_print('Log DIR:', args.log_dir)
formatted_print('Result DIR:', args.res_dir)
formatted_print('GAN TYPE:', args.gantype)

if args.multiprocessing_distributed:
    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
else:
    generator = main_worker(args.gpu, ngpus_per_node, args)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.


Total Number of GPUs:                   1                                       
Total Number of Workers:                8                                       
Batch Size:                             2                                       
Max image Size:                         224                                     
Min image Size:                         28                                      
Log DIR:                                drive/dl_projects/Video_Genaeration/output/./logs/SinGan_2019-11-20_23-47-25
Result DIR:                             drive/dl_projects/Video_Genaeration/output/./results/SinGan_2019-11-20_23-47-25
GAN TYPE:                               zerogp                                  
Use GPU: 0 for training
[28, 37, 49, 66, 88, 117, 157, 209]


Stage: [0/7] Avg Loss: D[1.333] G[6.504] RMSE[1.000]:  20%|██        | 4/20 [00:02<00:11,  1.41it/s]

DISCRIMINATOR LEARNING RATE UPDATE TO : 5e-05
GENERATOR LEARNING RATE UPDATE TO : 5e-05


Stage: [0/7] Avg Loss: D[1.212] G[2.629] RMSE[1.000]: 100%|██████████| 20/20 [00:03<00:00, 10.14it/s]


...

DISCRIMINATOR PROGRESSION DONE
GENERATOR PROGRESSION DONE


Stage: [7/7] Avg Loss: D[1.138] G[1.850] RMSE[0.076]:  20%|██        | 4/20 [00:03<00:17,  1.09s/it]

DISCRIMINATOR LEARNING RATE UPDATE TO : 5e-05
GENERATOR LEARNING RATE UPDATE TO : 5e-05


Stage: [7/7] Avg Loss: D[0.880] G[1.830] RMSE[0.079]: 100%|██████████| 20/20 [00:10<00:00,  2.37it/s]


DISCRIMINATOR PROGRESSION DONE
GENERATOR PROGRESSION DONE


In [0]:
data_loader = DataLoader(transformed_dataset, batch_size = args.batch_size,
                            sampler=torch.utils.data.SequentialSampler(transformed_dataset) ,  num_workers=args.workers)

In [0]:
def evaluate(stage, generator):
  train_it = iter(data_loader)

  x_in = next(train_it)

  x_in = x_in.cuda( non_blocking=True)
  x_org = x_in
  
  x_in = F.interpolate(x_in, (args.size_list[stage], args.size_list[stage]), mode='bilinear', align_corners=True)

  x_in_list = []
  print(len(args.size_list))
  for xidx in range(0, stage + 1):
      x_tmp = F.interpolate(x_org, (args.size_list[xidx], args.size_list[xidx]), mode='bilinear', align_corners=True)
      x_tmp = F.pad(x_tmp, [5, 5, 5, 5], value=0)
      x_in_list.append(x_tmp)
  generator.eval()
  out = generator(x_in_list)
  return out
  

In [69]:
for stage in range(args.stage, args.num_scale + 1):
  out = evaluate(stage,generator)
  networks =  generator
  generator.progress()
  

8


IndexError: ignored

In [0]:
w = torch.empty(500,300)
a=  torch.empty(500,300)
a= nn.init.kaiming_uniform_(a, mode='fan_in')



nn.init.kaiming_uniform_(w, mode='fan_in')
(a*w).argmax()

tensor(12876)

In [0]:
w = torch.rand(32,256,20,20)
w.argmax()

tensor(2207442)

In [0]:
a=torch.chunk(w,8,dim=1)

In [0]:
c=torch.reshape(w,(8,32,32,20,20))

In [0]:
d=torch.stack(a,dim=0)

In [0]:
torch.equal(c,d)

False

In [0]:
torch.all(torch.lt(torch.abs(torch.add(c, -d)), 1e-1))

tensor(False)

In [0]:
sz=4
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
print(mask)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

tensor([[ True, False, False, False],
        [ True,  True, False, False],
        [ True,  True,  True, False],
        [ True,  True,  True,  True]])


In [0]:
mask

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [0]:
mask = (torch.triu(torch.ones(sz, sz)) == 1)

In [0]:
def mult_4d( batch, q_w, bs):
  layers = batch.shape[1]
  dim1 = batch.shape[2]
  dim2 = batch.shape[3]
  output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
  return output.view(bs,layers,dim1,dim2)

In [0]:
a=torch.tensor([[1,2],[3,4]])
b=torch.tensor([[1,2],[3,4]])
b@a
#torch.bmm(a.unsqueeze(0),b.unsqueeze(0))

tensor([[ 7, 10],
        [15, 22]])

In [0]:
w = torch.rand(4,2,2,2)
a = torch.rand(4,2,2,2)
o=mult_4d(w,a.transpose(-2,-1),4)
#torch.bmm(w,a).shape
l=nn.Dropout()
l(o).shape

torch.Size([4, 2, 2, 2])

In [0]:
def generate_square_subsequent_mask(sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

mask= generate_square_subsequent_mask(4)
mask

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [0]:
mask.unsqueeze(0).unsqueeze(0)+a

RuntimeError: ignored

In [0]:
mask.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4, 4])

In [0]:
l=nn.Softmax2d()#.shape

In [0]:
c= l(mult_4d(w,a,w.shape[0]))

In [0]:
c.shape

torch.Size([4, 2, 2, 2])

In [0]:
l=nn.Softmax2d()#.shape

In [73]:


# For the current version: 
!pip install --upgrade tensorflow


     |████████████████████████████████| 86.3MB 86kB/s 
     |████████████████████████████████| 3.8MB 47.6MB/s 
     |████████████████████████████████| 450kB 51.9MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalle

In [0]:
import tensorflow as tf

In [0]:
transformed_dataset = CityscapeDataset(
                                           transforms=composed_transforms
                                           )

In [0]:
im_paths = transformed_dataset.im_path()

In [0]:
BUFFER_SIZE = 400
BATCH_SIZE = 32
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3

In [0]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)


  return input_image

In [0]:
def resize(input_image,  height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
 
  return input_image

def random_crop(input_image):
  stacked_image = tf.stack([input_image, input_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0]

# normalizing the images to [-1, 1]

def normalize(input_image):
  input_image = (input_image / 127.5) - 1


  return input_image

In [0]:

def random_jitter(input_image):
  # resizing to 286 x 286 x 3
  input_image = resize(input_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image = random_crop(input_image)


  return input_image

In [0]:
def load_image_train(image_file):
  input_image = load(image_file)
  input_image = random_jitter(input_image)


  return input_image

In [0]:
#This genrates batches of specified sizes, functions uded in this can fe find in other files in repo,(augmentation,preprocess)
#It can be upadatesd for genrating  batches of test files to
def image_generator(input_ids, batch_size = 32,is_test =False):
  
  while True:

    if is_test:
      batch_paths =input_ids[0:batch_size]
      for input_id in batch_paths:
      
        input = load_image_train(input_id)
            
        batch_input += [input]
      
      batch_x = tf.convert_to_tensor(batch_input)
      yield batch_input

    batch_paths = np.random.choice(a= input_ids, size = batch_size)
    
    batch_input = []
   
    
    for input_id in batch_paths:
      #print(input_id)
      
      input = load_image_train(input_id)
            
      batch_input += [input]
      
    
    batch_x = tf.convert_to_tensor(
    batch_input,
    dtype=None,
    dtype_hint=None,
    name=None
)
    #print(2)
    
    
    yield (batch_x, batch_x)
      

In [0]:
gen = image_generator(im_paths, 4)

In [0]:
x,y = next(gen)

In [181]:
x.shape

TensorShape([4, 256, 256, 3])

In [0]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [0]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [0]:
def Generator():
  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[256,256,3])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [0]:
generator = Generator()


In [0]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [0]:
discriminator = Discriminator()


In [0]:
LAMBDA = 100

In [0]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [0]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [0]:
checkpoint_dir = "drive/dl_projects/Video_Genaeration/pix"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [0]:
EPOCHS = 200

In [0]:
def generate_images(model, test_input):
  # the training=True is intentional here since
  # we want the batch statistics while running the model
  # on the test dataset. If we use training=False, we will get
  # the accumulated statistics learned from the training dataset
  # (which we don't want)
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], prediction[0]]
  title = ['ground Image',  'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [0]:

def train_step(input_image, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

In [0]:

import time

In [0]:
def train(dataset, epochs):
  print(1)
  for epoch in range(epochs):
    start = time.time()

    input_image , target = next(dataset)
    train_step(input_image, target)

    

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))

In [0]:

gen = image_generator(im_paths, 4)

In [200]:
x,y=(next(gen))
x.shape

TensorShape([4, 256, 256, 3])

In [0]:
EPOCHS = 200
train((gen), EPOCHS)

1
Time taken for epoch 1 is 12.370551347732544 sec



In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))